## Insertando un documento

Usa alguna de las formas de conectarse a la DB de Mongo Atlas(nube) en este caso desde VSCode. Creamos una nueva carpeta *04-insert-doc* y dentro un archivo .mongodb

Vamos a crear una nueva BD. No es necesario crearla explicitamente, simplemente si le decimos vamos a usar tal base de datos y esta no esta creada, el inmediatamtente la crea. Lo mismo para las colecciones.

La primera instruccion para insertar un documento es *insertOne*

    use("platzi_store")
    db.products.insertOne({
        name: "Licuadora",
        price: 1100
    })

    db.products.insertOne({
        name: "Microondas",
        price: 360
    })

Al correr el documento, sale:

![](https://i.imgur.com/McWuZYY.png)

Como si solo hubiera insertado uno solo,SIgnigica el *playground* que estamos corriendo se queda solo con la ultima instruccion.

Pero si revisamos encontraremos efectivamente estan los dos docs. 

    db.products.find().count()

Al intentar usar el comando *show collections* en VScode no lo deja hacer, pero desde la *MongoSh* si.

    show collections


Observa esta consulta:

    [
    {
        "_id": {
        "$oid": "640b9e7fc3aa787c39b067be"
        },
        "name": "Licuadora",
        "price": 1100
    },
    {
        "_id": {
        "$oid": "640b9e7fc3aa787c39b067bf"
        },
        "name": "Microondas",
        "price": 360
    }
    ]

*oid* es object id y es unico y asignado directamete por mongo, igualmente tu puedes asignar tus propios IDs, pero ten cuidado.

    db.products.insertOne({
        _id: 1,
        name: "tostadora",
        price: "341"    
    })

## Insertando varios documentos

Crearemos una nueva carpeta llamada *05-insert-many* y dentro el archivo *products.mongodb*. Usar la siguiente funcion con cuidado para borrar todos los documentos de una coleccion:

    use("platzi_store")
    db.products.drop()

Al parecer como esta BD de datos solo contenia una coleccion cuyos documentos fueron borrados en su totalidad, al consultar con el comando *show dbs* desde mongo shell, pues *platzi_store* tambien fue borrada. Igualmente este no es un problema por ahora.

Al usar la instruccion *insertMany* se usa la estructura de un array:

        db.products.insertMany([{
            name: "ducha",
            price: 540
        },
        {
            name: "horno",
            price: 1200
        },
        {
            name: "nevera",
            price: 980
        },
        ])


### ¿QUe pasa si tenemos ids duplicados?

volvamos a realizar el ejercicio esta vez asignando manualmente los IDs.

    db.products.insertMany([{
        _id : 1,
        name: "ducha",
        price: 540
    },
    {
        _id : 2,
        name: "horno",
        price: 1200
    },
    {
        _id : 1,
        name: "nevera",
        price: 980
    },
    {
        _id : 4,
        name: "plancha",
        price: 120
    },
    ])


En este caso, saldra un error de duplicacion, sin embargo insertara los dos primeros documentos. ¿COmo podriamos entonces dejar solo por fuera los que tienen error de duplicacion, pero los demas no?

Con la instruccion *ordered=false*. Pero a mi no me funciona

## Actualizando un documento

### Desde Mongo Atlas

Directamente desde la interfax grafica

### Con Comandos

Crear una nueva carpeta *06-update-doc* y dentro el respectivo playground

#### updateOne

Se debe hacer uso de los *operadores atomicos*. Por ejemplo, si queremos cambiar el nombre del documento con ID = 2, el *atomic operator* es *$set*.

La primera parte corresponde al *query* y la segunda parte a la respectiva actualizacion.

    db.products.updateOne(
    // query
    {
        _id:2},
    {
        /**
         * field: The field name
         * expression: The expression.
         */
        $set: {
          name: "horno magico"
        }
    }
    )

##### Ejemplo. 

Actualizar el documento correspondiente al de la nevera, y cambiar su nombre y precio:

    use("platzi_store")
    
    db.products.updateOne(
        // query
        {
            name:"nevera"},
        {

            $set: {
            name: "nevera magica",
            price: 1234
            }
        }
    )

##### Ejemplo2.

Actualizar el documento correspondiente a la ducha, y cambiar nombre y precio, y ademas agregar un nuevo atributo llamado tags.

    db.products.updateOne(
        {
            name:"ducha"
        },
        {
            $set: {
            name: "ducha magica",
            price: 56789,
            tags: ['A', 'B', 'C'],
            }
        }
    )

#### Operador $inc

al igual que $set, otro operador es $inc que es para incrementar atributos numericos. El valor actual mas el incremento. Tambien se pueden incrementos negativos.

    db.products.updateOne(
    {
        _id:4
    },
    {

        $inc: {
          price: 240,
        }
    })


#### Funcion ObjectId

Para los IDs generados autogenerados, se necesita dicha funcion para el query. 

    db.products.updateOne(
    {
        _id: ObjectId("640e4b1fb946da9d6a2d47e9")
    },
    {
        $inc: {
            price:1000,
        }
    }

    )